# Tik Tok Crawler

This program crawls and logs into tik tok via facebook using chromedriver web browser. The crawler will go to a hastag selected and scrape content from the most popular 30 videos under that tag. It then scrapes all content off the page besides the video, going to replies that are 4 levels deep.

Caption, likes, comments, usernames, replies to comments, and hastags are scraped.

It writes all content in list form to a json file which separates the posts indicated by "New Post"

In [ ]:
'''
@author osamah Abdelhaq last edit: 6/12/2020 
'''
#imports
from time import sleep
from selenium import webdriver
from getpass import getpass
import json
from selenium.webdriver.chrome.options import Options
import requests

In [ ]:
class Bot:
    def __init__(self, username, password, url): 
        
        #can add headless option if preferred, this is not headless for debugging purposes
        
        self.url = url
        #initiate browser
        self.opts = Options()
        self.opts.add_argument("user-agent=Googlebot")
        
#********************change chromdriver executable_path********************
        self.browser = webdriver.Chrome(executable_path="C:\chromedriver_win32\chromedriver.exe",options=self.opts)
        
        
        self.browser.implicitly_wait(5)
        self.browser.get('https://www.tiktok.com/login/?redirect_url=https%3A%2F%2Fwww.tiktok.com%2Ftrending%2F%3Flang%3Dfr&lang=en&enter_method=top_bar')
        sleep(2)
        self.browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[2]/div[2]/div[2]').click()
        sleep(2)

        # switch window
        root = self.browser.window_handles[0]
        self.browser.switch_to_window(self.browser.window_handles[1]) # switch to popup facebook login window

        # login with facebook
        self.browser.find_element_by_xpath('//*[@id="email"]').send_keys(username)
        self.browser.find_element_by_xpath('//*[@id="pass"]').send_keys(password)
        self.browser.find_element_by_xpath('//*[@id="u_0_0"]').click()
        sleep(10)

        #switch to first window
        self.browser.switch_to_window(root) 

        sleep(2)
        self.browser.get("https://www.tiktok.com/trending/?lang=en&loginType=facebook")
        
        #===================url of tag===================
        self.browser.get(self.url)
        
        sleep(2)
        
    def getContent(self):
        
        #get links to all Tik Tok videos in the tag
        video_links = []
        
        #this finds all elements with an href. An href is a link to the video.
        ids = self.browser.find_elements_by_xpath("//*[@href]")

        #retreive the href attribute and store it
        for h in ids:
            link = h.get_attribute('href')
            if ("?lang=en" and "/video/" in link):
                video_links.append(link)
        
        all_comments = []
        
        
        for vid in video_links:
            #get each video and scrape all data on the page besides the video content itself
            sleep(2)
            self.browser.get(vid)
            sleep(2)

            #Extended loop clicks to expand replies in relevant comments

            #finds all clickable buttons of this class name then attempts to click to expand
            replies = self.browser.find_elements_by_class_name("jsx-2071094126")
            
            for reply in replies:
                try:
                    if "View replies" in reply.text:
                        reply.click()
                        sleep(0.5)
                        
                        try:
                            for get in range(4): #clicks view more comments four times
                                
                                #find elements that start have a class name that starts with 'more'
                                more_replies = self.browser.find_elements_by_css_selector("p[class^='more']")
                                
                                for more in more_replies:
                                    try:
                                        more.click()
                                    except:
                                        pass
                        except:
                            pass
                except:
                    pass

            #the first element that starts with 'jsx' has the entire content of that page
            #includes likes, comments, usernames, dates comments were posted, and any associated hashtags
            objects = self.browser.find_element_by_css_selector("div[class^='jsx']")
            comments = objects.text
            all_comments.append("\n**New Post**\n")
            all_comments.append(comments)
            
        return all_comments
        
    def saveData(self,all_comments):
#*********************Change Storage path if preferred********************
        # save the data
        with open('Tik_Tok_Stimulus_check', 'w') as out_f:
            json.dump(all_comments, out_f)
        self.browser.close()
        return

In [ ]:
#==========================================================================
def main():
    username = input("Username: ") 
    password = getpass("password: ")

    #url of the tag to scrape
    hashtag = "https://www.tiktok.com/tag/stimuluscheck?lang=en&loginType=facebook"
    
    #initiate instance
    tiktokBot = Bot(username, password,hashtag)
    
    #get content from most popular 30 videos
    all_content = tiktokBot.getContent()
    
    #save content in json format
    tiktokBot.saveData(all_content)

    sleep(50)
#==========================================================================
    
if __name__ == "__main__":
    main()